# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 6.6598e-01, -1.1734e+00],
         [-1.0172e+00,  8.5356e-01],
         [ 5.1016e-01, -3.7092e-01],
         [-7.4688e-01,  9.4767e-01],
         [-7.2595e-01,  2.9072e-04],
         [ 1.1870e-01,  1.0007e+00],
         [ 1.5617e+00, -7.3586e-01],
         [-1.3379e+00,  3.0572e-01],
         [-7.5865e-01,  3.7648e-02],
         [ 2.3923e-01, -9.9570e-01]]),
 tensor([[ 9.5120],
         [-0.7282],
         [ 6.4605],
         [-0.5220],
         [ 2.7453],
         [ 1.0332],
         [ 9.8274],
         [ 0.4862],
         [ 2.5614],
         [ 8.0553]])]

Use a framework's predefined layers

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Initialize the model parameters

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm

In [8]:
loss = nn.MSELoss()

Instantiate an `SGD` instance

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

The training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000237
epoch 2, loss 0.000105
epoch 3, loss 0.000104


Compare the model parameters learned by training on finite data
and the actual parameters

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-2.8372e-05, -6.5899e-04])
error in estimating b: tensor([-0.0003])
